# О том, как учат сложные сетки 

In [1]:
import numpy as np
import tensorflow as tf
tf.__version__ 

'2.1.0'

In [2]:
from tensorflow.keras import Sequential, Model
keras, L = tf.keras, tf.keras.layers

В этой тетрадке немного поработаем с градусами по цельсию и фаренгейту! Снова попробуем восстановить формулу 

$$ f = c \times 1.8 + 32 $$

In [3]:
celsius    = np.array([-40, -10,  0,  8, 15, 22,  38],  dtype=float)
fahrenheit = np.array([-40,  14, 32, 46, 59, 72, 100],  dtype=float)

for i,c in enumerate(celsius):
    print("{} degrees Celsius = {} degrees Fahrenheit".format(c, fahrenheit[i]))

-40.0 degrees Celsius = -40.0 degrees Fahrenheit
-10.0 degrees Celsius = 14.0 degrees Fahrenheit
0.0 degrees Celsius = 32.0 degrees Fahrenheit
8.0 degrees Celsius = 46.0 degrees Fahrenheit
15.0 degrees Celsius = 59.0 degrees Fahrenheit
22.0 degrees Celsius = 72.0 degrees Fahrenheit
38.0 degrees Celsius = 100.0 degrees Fahrenheit


In [4]:
# транспонировали выборку
x_train = celsius[:,None]
y_train = fahrenheit[:,None]

# 1. Как мы обучались до этого 

На Keras было всё совсем просто. 

In [5]:
model = Sequential()
model.add(L.Dense(1))

opt = tf.keras.optimizers.Adam( )

model.compile(loss='mse', optimizer=opt)
model.fit(x_train, y_train, validation_split=0.2, epochs=2, verbose=1)

Train on 5 samples, validate on 2 samples
Epoch 1/2
5/5 [==============================] - 0s 46ms/sample - loss: 1914.4365 - val_loss: 8692.5264
Epoch 2/2
5/5 [==============================] - 0s 1ms/sample - loss: 1913.1516 - val_loss: 8686.5801


После мы сказали, что иногда хочется собирать более сложные модели, а ещё жёстко контролировать обучение, и посмотрели на Tensorflow.

In [6]:
a = tf.Variable(tf.random.normal([1]))
b = tf.Variable(tf.random.normal([1]))

# Наша модель
def linear_regression(x):
    return a + b*x

# Ошибка для модели
def mean_square(y_pred, y_true):
    return tf.reduce_mean((y_pred-y_true)**2)

# оптимизатор 
optimizer = tf.optimizers.SGD(learning_rate=0.001)

# процесс оптимизации
def model_train(X, Y):

    # находим loss и пробрасываем градиент
    with tf.GradientTape() as g:
        pred = linear_regression(X)
        loss = mean_square(pred, Y)

    # Вычисляем градиенты
    gradients = g.gradient(loss, [a, b])
    
    # Обновляем веса a и b в ходе одной итерации спуска 
    optimizer.apply_gradients(zip(gradients, [a, b]))
    pass

In [7]:
#Обучение
epochs = 100 # число эпох 

for i in range(epochs):
    
    # Делаем щаг градиентного спуска 
    model_train(celsius, fahrenheit)
    
    # Каждую сотую итерацию следим за тем, что произошло
    if i%10 == 0:
        y_pred = linear_regression(celsius)
        loss_val = mean_square(y_pred, fahrenheit)
        print("step: %i, loss: %f, a: %f, b: %f" % (i, loss_val, a.numpy(), b.numpy()))

step: 0, loss: 1040.500732, a: -0.090210, b: 2.379928
step: 10, loss: 948.990540, a: 0.517278, b: 2.063241
step: 20, loss: 913.200867, a: 1.115780, b: 2.058189
step: 30, loss: 878.761108, a: 1.702886, b: 2.053235
step: 40, loss: 845.620239, a: 2.278815, b: 2.048374
step: 50, loss: 813.729370, a: 2.843778, b: 2.043607
step: 60, loss: 783.041138, a: 3.397985, b: 2.038929
step: 70, loss: 753.510559, a: 3.941639, b: 2.034341
step: 80, loss: 725.093628, a: 4.474943, b: 2.029841
step: 90, loss: 697.748535, a: 4.998094, b: 2.025425


# 2. Как обычно обучаются на Tensorflow 

Для того, чтобы было удобнее, модели заворачивают в полноценные классы. 

In [8]:
tf.keras.backend.set_floatx('float64')
class Super_puper_neural_net(Model):
    
    def __init__(self, n_hidden_neurons):
        super(Super_puper_neural_net, self).__init__()
        self.fc1 = L.Dense(n_hidden_neurons, kernel_initializer='glorot_uniform',
                           activation='relu', trainable=True)
        self.fc2 = L.Dense(n_hidden_neurons, kernel_initializer='glorot_uniform',
                           trainable=True)

    def encode(self, x):
        x = self.fc1(x)
        x = self.fc2(x)
        return x

In [9]:
model = Super_puper_neural_net(1)
model.encode(x_train)

<tf.Tensor: shape=(7, 1), dtype=float64, numpy=
array([[  0.        ],
       [  0.        ],
       [  0.        ],
       [-14.08335521],
       [-26.40629101],
       [-38.72922682],
       [-66.89593723]])>

In [10]:
model1 = Super_puper_neural_net(1)
model2 = Super_puper_neural_net(1)

x1 = model1.encode(x_train)
x2 = model2.encode(x_train)

In [11]:
# Список из переменных
model.variables

[<tf.Variable 'dense_1/kernel:0' shape=(1, 1) dtype=float64, numpy=array([[1.48600645]])>,
 <tf.Variable 'dense_1/bias:0' shape=(1,) dtype=float64, numpy=array([0.])>,
 <tf.Variable 'dense_2/kernel:0' shape=(1, 1) dtype=float64, numpy=array([[-1.18466471]])>,
 <tf.Variable 'dense_2/bias:0' shape=(1,) dtype=float64, numpy=array([0.])>]

Можно легко подменять одни переменные какими-нибудь своими. Это довольно удобно, когда хочется позаимствовать чужие веса. В будущем мы будем этим активно заниматься. 

In [12]:
# Ошибка для модели
def mean_square(y_pred, y_true):
    return tf.reduce_mean((y_pred-y_true)**2)

# оптимизатор 
optimizer = tf.optimizers.SGD(learning_rate=0.001)

# процесс оптимизации
def model_train(X, Y):

    # находим loss и пробрасываем градиент
    with tf.GradientTape() as g:
        pred = model.encode(X)
        loss = mean_square(pred, Y)

    # Вычисляем градиенты
    gradients = g.gradient(loss, model.variables)
    
    # Обновляем веса a и b в ходе одной итерации спуска 
    optimizer.apply_gradients(zip(gradients, model.variables))


In [13]:
#Обучение
epochs = 100 # число эпох 

for i in range(epochs):
    
    # Делаем щаг градиентного спуска 
    model_train(x_train, y_train)
    
    # Каждую сотую итерацию следим за тем, что произошло
    if i%10 == 0:
        y_pred = model.encode(x_train)
        loss_val = mean_square(y_pred, y_train)
        print("step: %i, loss: %f" % (i, loss_val))

step: 0, loss: 17571.846658
step: 10, loss: nan
step: 20, loss: nan
step: 30, loss: nan
step: 40, loss: nan
step: 50, loss: nan
step: 60, loss: nan
step: 70, loss: nan
step: 80, loss: nan
step: 90, loss: nan


# 3. Свой слой на Tensorflow для Keras

Новые слои можно писать на основе керасовского класса `Layer`. Если прописать `help(tf.keras.layers.Layer)`, можно почитать про него. Если в кратце, нужно реализовать три части: 

* Конструктор, в нём мы описываем гиперпараметры 
* Метод `build`, в которм мы описываем все переменные 
* Метод `call`, который делает forward pass

In [14]:
class MyLinear(L.Layer):
    
    # Задаём консруктор 
    def __init__(self, units=32):
        super(MyLinear, self).__init__()  # чтобы коректно унаследовались методы
        self.units = units                # число нейронов

    def build(self, input_shape):
        # add_weight внутри build то же самое что и Variable, но совместимо с Keras
        self.w = self.add_weight(shape=(input_shape[-1], self.units),
                                 initializer='random_normal', 
                                 trainable=True)
        
        self.b = self.add_weight(shape=(self.units,),
                                 initializer='random_normal', 
                                 trainable=True)

    # Применение 
    def call(self, inputs):
        # сразу делаем и линейное преобразование и ReLU (а почему бы и нет)
        return tf.nn.relu(tf.matmul(inputs, self.w) + self.b) 

In [15]:
model = Sequential()
model.add(MyLinear())  # добавили свой слой! 
model.add(L.Dense(1))

In [16]:
opt = tf.keras.optimizers.Adam( )
model.compile(loss='mse', optimizer=opt)

In [17]:
model.fit(x_train, y_train, validation_split=0.2, epochs=2, verbose=1)

Train on 5 samples, validate on 2 samples
Epoch 1/2
5/5 [==============================] - 0s 53ms/sample - loss: 1740.3874 - val_loss: 7942.8867
Epoch 2/2
5/5 [==============================] - 0s 1ms/sample - loss: 1735.8881 - val_loss: 7928.0667


По аналогии с помощью класса `Model` можно делать полноценные слои-модели. Этим мы тоже будем позже заниматься. Почитать про свои слои и модели можно подробнее [вот тут, в документации.](https://www.tensorflow.org/beta/guide/keras/custom_layers_and_models)